In [1]:
import tensorflow as tf
import keras as ks
# general
import os
import sys
import io

import argparse
from datetime import datetime
import h5py
import importlib

# Indispensables
import pandas as pd
import numpy as np
import geopandas as gpd
import math

# VISU
from IPython.display import Image, IFrame
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm


import colorsys
print('TensorFlow: %s' % tf.__version__)
print('Kerasw: %s' % ks.__version__)
print('Panda version: %s' % pd.__version__)
print('Numpy version: %s' % np.__version__)
print('Geo Panda version: %s' % gpd.__version__)


Using TensorFlow backend.


TensorFlow: 2.0.0
Kerasw: 2.3.1
Panda version: 0.25.1
Numpy version: 1.16.5
Geo Panda version: 0.6.3


In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = "S:\\Commun\\SIG3D\\2020\\PROJETS_2020\\20039_MachineLearning\\DATA\\"
DALLE_NUM = DATA_DIR + "DallesNumPoints5m"
DALLE_ECA = DATA_DIR + "DallesEcartZ5m"
DALLE_POS = DATA_DIR + "DallesPosition5m"
print("Working directory:",BASE_DIR)
print("Data ROOT directory:",DATA_DIR)

sys.path.append(DATA_DIR)
sys.path.append(os.path.join(DATA_DIR,'DallesNumPoints5m\\'))
sys.path

Working directory: C:\Users\STG3841\Bureau (local)
Data ROOT directory: S:\Commun\SIG3D\2020\PROJETS_2020\20039_MachineLearning\DATA\


['C:\\Users\\STG3841\\Bureau (local)',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\python37.zip',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3',
 '',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\STG3841\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\STG3841\\.ipython',
 'S:\\Commun\\SIG3D\\2020\\PROJETS_2020\\20039_MachineLearning\\DATA\\',
 'S:\\Commun\\SIG3D\\2020\\PROJETS_2020\\20039_MachineLearning\\DATA\\DallesNumPoints5m\\']

In [3]:
from os import listdir
#from os.path import isfile, join
#from os import walk
import glob

#liste1 = [f for f in listdir(DALLE_NUM) if isfile(join(DALLE_NUM, f))]
liste_1 = [f for f in glob.glob(DALLE_NUM + "\\*.tif", recursive=True)][:2]   #only first 10 for test
liste_2 = [f for f in glob.glob(DALLE_ECA + "\\*.tif", recursive=True)][:2]  
liste_3 = [f for f in glob.glob(DALLE_POS + "\\*.tif", recursive=True)][:2]   
print(len(listdir(DALLE_NUM)), len(liste_1))

liste_NUM = os.listdir(DALLE_NUM)
liste_ECA = os.listdir(DALLE_ECA)
liste_POS = os.listdir(DALLE_POS)
print(len(liste_NUM), len(liste_ECA)/2, len(liste_POS)/2)

22287 2
22289 447.0 11143.0


In [4]:
from PIL import Image
import rasterio
import imageio

fichier = liste_1[0]
print(fichier)
im = imageio.imread(fichier)
print(im.shape)
df = pd.DataFrame(im)
df

S:\Commun\SIG3D\2020\PROJETS_2020\20039_MachineLearning\DATA\DallesNumPoints5m\2042500_7272000_NumPoints5m.tif
(10, 10)


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,3.0,0.0
2,0.0,0.0,0.0,0.0,1.0,3.0,3.0,5.0,2.0,1.0
3,0.0,0.0,0.0,0.0,6.0,2.0,4.0,2.0,1.0,2.0
4,0.0,0.0,0.0,1.0,4.0,1.0,2.0,2.0,2.0,1.0
5,0.0,0.0,0.0,1.0,3.0,4.0,5.0,1.0,1.0,3.0
6,0.0,1.0,5.0,7.0,4.0,3.0,1.0,2.0,0.0,4.0
7,2.0,0.0,1.0,2.0,3.0,1.0,2.0,3.0,3.0,0.0
8,2.0,2.0,1.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0
9,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
fich_lamp = DATA_DIR+"LampadairePresence_0_1_5m.asc"
header = pd.read_csv(fich_lamp, sep=" ", header=None, skiprows=0, nrows=5, names=["noms","valeurs"])
n_row =  header['valeurs'][0]
n_col =  header['valeurs'][1]
xlow = header['valeurs'][2]
ylow = header['valeurs'][3]
cell_size = header['valeurs'][4]
#ylow = pd.read_csv(fich_lamp, sep=" ", skiprows=3, header=None, nrows=1)[1]
print("Bord inférieur gauche:", xlow, ylow, n_row, n_col, cell_size)

lampadaires = pd.read_csv(fich_lamp, sep=" ", header=None, skiprows=6)
#print(lampadaires.count())
print(lampadaires[0].value_counts())
print(lampadaires.columns)
#print(lampadaires.nunique())
print(lampadaires.nunique().agg(['mean','count','sum']))

# print(lampadaires.describe())
lampes = pd.DataFrame(np.reshape(lampadaires.to_numpy(), n_row*n_col))
print(lampes.shape)

Bord inférieur gauche: 2042500 7272000 200 200 5
0.0    197
1.0      3
Name: 0, dtype: int64
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            190, 191, 192, 193, 194, 195, 196, 197, 198, 199],
           dtype='int64', length=200)
mean       1.79
count    200.00
sum      358.00
dtype: float64
(40000, 1)


In [ ]:
lampes['Num0'] = lampes.index
lampes['Num'] = lampes['Num0'].mod(other=n_col)
lampes['Num1'] = (lampes['Num0'] - n_row + 1) / n_row
lampes['Num2'] = lampes['Num1'].apply(np.ceil)

lampes.rename(columns={0: 'present'}, inplace=True)
#lampes['X'] = xlow + np.round((lampes['Num0'] - n_row + 1)/n_row, 0) * cell_size
lampes['X'] = xlow + lampes['Num2'] * cell_size
lampes['Y'] = ylow + lampes['Num'] * cell_size
#lampes['X'] = xlow + lampes.index + (n_row - lampes.index) * n_col
#print(lampes.describe(include='all'))
print(lampes.sum())

print(lampes.describe(include='all'))
lampes['nom_ECA'] = str(lampes['X']) + "_" + str(lampes['Y']) + "_EcartZ5m.tif"
lampes['nom_POS'] = str(lampes['X']) + "_" + str(lampes['Y']) + "_Position5m.tif"


In [ ]:
lampes

In [8]:
lampes.head(2)
lampes.nunique()


present        2
Num0       40000
Num          200
Num1       40000
Num2         200
X            200
Y            200
nom_ECA        1
dtype: int64

In [9]:
# Read files in subdirectories
#liste_f = ["liste_1", "liste_2", "liste_3"]
for n in range(0,len(liste_1)):
#for n in range(0,1):
    #for i in range(0,len(liste_f)):
    #with open(liste_f[n]) as fichierNum:
    with open(liste_3[n]):
        fichierNum3 = liste_3[n]
        df_3a = imageio.imread(fichierNum3)
        df_3 = pd.DataFrame(df_3a)
    print(df_3a.shape, "\n", df_3)
    
    with open(liste_1[n]):         # as fichierNum1:
        fichierNum1 = liste_1[n]
        df_1a = imageio.imread(fichierNum1)
        df_1 = pd.DataFrame(df_1a)
    print(df_1a.shape, "\n", df_1)
    with open(liste_2[n]):         # as fichierNum2:
        fichierNum2 = liste_2[n]
        print(fichierNum2)
        df_2a = imageio.imread(fichierNum2)
        df_2 = pd.DataFrame(df_2a)
    print(df_2a.shape, "\n", df_2)


(10, 10) 
      0    1    2    3    4    5    6    7    8    9
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
(10, 10) 
      0    1    2    3    4    5    6    7    8    9
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  6.0  3.0  1.0
1  0.0  0.0  0.0  0.0  0.0  0.0  4.0  3.0  3.0  0.0
2  0.0  0.0  0.0  0.0  1.0  3.0  3.0  5.0  2.0  1.0
3  0.0  0.0  0.0  0.0  6.0  2.0  4.0  2.0  1.0  2.0
4  0.0  0.0  0.0  1.0  4.0  1.0  2.0  2.0  2.0  1.0
5  0.0  0.0  0.0  1.0  3.0  4.0  5.0  1.0  1.0  3.0
6  0.0  1.0  5.0  7.0  4.0  3.0  1.0  2.0 